In [8]:
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from pygame import mixer

In [9]:
#detect the face
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

#in the detected face, detect the eyes
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

#call the model build from models directory/folder
model = load_model(r'D:\Driver-Drowsiness-Detection-System\models\model.h5')

In [19]:
mixer.init()
sound = mixer.Sound(r'D:\Driver-Drowsiness-Detection-System\alarm.wav')
cap = cv2.VideoCapture(0)  #you can give path to a .mp4 instead of live video capture
score = 0

while True:
    ret, frame = cap.read() #while video is running, return the frame to the program
    height, width = frame.shape[0:2]
    #pre-processing step
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #convert the frame to gray scale to predict better
    faces =face_cascade.detectMultiScale(gray, scaleFactor = 1.2, minNeighbors=3)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor = 1.1, minNeighbors=1)
    
    #black box of scores
    cv2.rectangle(frame, (0,height-50), (200,height), (0,0,0), thickness=cv2.FILLED)
    
    #draw rectangle over the face and eyes
    for (x,y,w,h) in faces:
        cv2.rectangle(frame, pt1=(x,y), pt2=(x+w,y+h), color=(255,0,0), thickness=3)
        
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(frame, pt1=(ex,ey), pt2=(ex+ew,ey+eh), color=(255,0,0), thickness=3)
        
        #preprocessing steps
        eye = frame[ey:ey+eh, ex:ex+w]
        eye = cv2.resize(eye,(80,80)) #resize
        eye = eye/255                 #rescale
        eye = eye.reshape(80,80,3)    #reshape
        eye = np.expand_dims(eye, axis=0)
        
        #model prediction
        prediction = model.predict(eye)
        
        #if eyes are closed, buzz the alarm
        if prediction[0][0] > 0.30:
            cv2.putText(frame, 'Close', (10,height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, 
                        fontScale=1, color=(255,255,255), thickness=1, lineType=cv2.LINE_AA)
            cv2.putText(frame, 'Score'+str(score), (100,height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, 
                        fontScale=1, color=(255,255,255), thickness=1, lineType=cv2.LINE_AA)
            score=score+1
            if (score>5):
                try:
                    sound.play()
                except:
                    pass
            
        #if eyes are open
        elif prediction[0][0] < 0.90:
            cv2.putText(frame, 'Open', (10,height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, 
                        fontScale=1, color=(255,255,255), thickness=1, lineType=cv2.LINE_AA)
            cv2.putText(frame, 'Score'+str(score), (100,height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, 
                        fontScale=1, color=(255,255,255), thickness=1, lineType=cv2.LINE_AA)
            score=score-1
            if (score<0):
                score=0
        
    #show the results
    cv2.imshow('frame', frame)
    if cv2.waitKey(33) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 37ms/step


1/1 [==============================] - 0s 52ms/step


1/1 [==============================] - 0s 40ms/step


KeyboardInterrupt: 